## DCN V2 Baseline

https://www.kaggle.com/code/nadare/tensorflow-baseline-lb-0-681

### Configuration

In [9]:
# モジュールの動的import(import先のファイルが更新されたときに追従する)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import os
from glob import glob
import gc

import numpy as np
import polars as pl
import pandas as pd
import tensorflow as tf

from src.utils.pipeline import Pipeline
from src.utils.aggregator import Aggregator


In [11]:
tf.version.VERSION

'2.12.0'

In [12]:
ROOT = "/kaggle"
TRAIN_DIR = os.path.join(ROOT, "input", "parquet_files", "train")
TEST_DIR = os.path.join(ROOT, "input", "parquet_files", "test")

### Read data

In [13]:
def read_file(path, depth=None):
  df = pl.read_parquet(path)
  df = df.pipe(Pipeline.set_table_dtypes)

  # depth>0 は集計値を使わないと、left join時にOOMになる（RAM 64GBでも)
  if depth in [1, 2]:
    df = df.group_by("case_id").agg(Aggregator.get_exprs(df))

  return df

def read_files(regex_path, depth=None):
  chunks = []
  for path in glob(str(regex_path)):
    chunks.append(pl.read_parquet(path).pipe(Pipeline.set_table_dtypes))

  df = pl.concat(chunks, how="vertical_relaxed")

  # depth>0 は集計値を使わないと、left join時にOOMになる（RAM 64GBでも)
  if depth in [1, 2]:
    df = df.group_by("case_id").agg(Aggregator.get_exprs(df))

  return df

In [14]:
data_store = {
  "base": read_file(os.path.join(TRAIN_DIR, "train_base.parquet")),
  "depth_0": [
      read_file(os.path.join(TRAIN_DIR, "train_static_cb_0.parquet")),
      read_files(os.path.join(TRAIN_DIR, "train_static_0_*.parquet")),
  ],
  "depth_1": [
      read_files(os.path.join(TRAIN_DIR, "train_applprev_1_*.parquet"), 1),
      read_file(os.path.join(TRAIN_DIR, "train_tax_registry_a_1.parquet"), 1),
      read_file(os.path.join(TRAIN_DIR, "train_tax_registry_b_1.parquet"), 1),
      read_file(os.path.join(TRAIN_DIR, "train_tax_registry_c_1.parquet"), 1),
      read_file(os.path.join(TRAIN_DIR, "train_credit_bureau_b_1.parquet"), 1),
      read_file(os.path.join(TRAIN_DIR, "train_other_1.parquet"), 1),
      read_file(os.path.join(TRAIN_DIR, "train_person_1.parquet"), 1),
      read_file(os.path.join(TRAIN_DIR, "train_deposit_1.parquet"), 1),
      read_file(os.path.join(TRAIN_DIR, "train_debitcard_1.parquet"), 1),
  ],
  "depth_2": [
      read_file(os.path.join(TRAIN_DIR, "train_credit_bureau_b_2.parquet"), 2),
  ],
}

In [17]:
def feature_eng(base, depth_0, depth_1, depth_2):
  base_df = (
    base.with_columns(
      month_decision=pl.col("date_decision").dt.month(),
      weekday_decision=pl.col("date_decision").dt.weekday(),
    )
  )

  # 全てのdfをleft join
  for i, df in enumerate(depth_0+depth_1+depth_2):
    base_df = base_df.join(df, how="left", on="case_id", suffix=f"_{i}")

  base_df = base_df.pipe(Pipeline.handle_dates).to_pandas()
  base_df = base_df.pipe(Pipeline.reduce_memory)

  return base_df

In [18]:
train_df = feature_eng(**data_store)

print(f"train data shape: {train_df.shape}")

del data_store
gc.collect()


start - end memory:- 4357.62 - 2649.80 Mb
train data shape: (1526659, 376)


266

In [19]:
pd.set_option('display.max_columns', 500)
train_df

,case_id,WEEK_NUM,target,month_decision,weekday_decision,assignmentdate_238D,assignmentdate_4527235D,assignmentdate_4955616D,birthdate_574D,contractssum_5085716L,dateofbirth_337D,dateofbirth_342D,days120_123L,days180_256L,days30_165L,days360_512L,days90_310L,description_5085714M,education_1103M,education_88M,firstquarter_103L,for3years_128L,for3years_504L,for3years_584L,formonth_118L,formonth_206L,formonth_535L,forquarter_1017L,forquarter_462L,forquarter_634L,fortoday_1092L,forweek_1077L,forweek_528L,forweek_601L,foryear_618L,foryear_818L,foryear_850L,fourthquarter_440L,maritalst_385M,maritalst_893M,numberofqueries_373L,pmtaverage_3A,pmtaverage_4527227A,pmtaverage_4955615A,pmtcount_4527229L,pmtcount_4955617L,pmtcount_693L,pmtscount_423L,pmtssum_45A,requesttype_4525192L,responsedate_1012D,responsedate_4527233D,responsedate_4917613D,riskassesment_302T,riskassesment_940T,secondquarter_766L,thirdquarter_1082L,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,avgdbddpdlast24m_3658932P,avgdbddpdlast3m_4187120P,avgdbdtollast24m_4525197P,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,avglnamtstart24m_4525187A,avgmaxdpdlast9m_3716943P,avgoutstandbalancel6m_4187114A,avgpmtlast12m_4525200A,bankacctype_710L,cardtype_51L,clientscnt12m_3712952L,clientscnt3m_3712950L,clientscnt6m_3712949L,clientscnt_100L,clientscnt_1022L,clientscnt_1071L,clientscnt_1130L,clientscnt_136L,clientscnt_157L,clientscnt_257L,clientscnt_304L,clientscnt_360L,clientscnt_493L,clientscnt_533L,clientscnt_887L,clientscnt_946L,cntincpaycont9m_3716944L,cntpmts24_3658933L,commnoinclast6m_3546845L,credamount_770A,credtype_322L,currdebt_22A,currdebtcredtyperange_828A,datefirstoffer_1144D,datelastinstal40dpd_247D,datelastunpaid_3546854D,daysoverduetolerancedd_3976961L,deferredmnthsnum_166L,disbursedcredamount_1113A,disbursementtype_67L,downpmt_116A,dtlastpmtallstes_4499206D,eir_270L,equalitydataagreement_891L,equalityempfrom_62L,firstclxcampaign_1125D,firstdatedue_489D,homephncnt_628L,inittransactionamount_650A,inittransactioncode_186L,interestrate_311L,interestrategrace_34L,isbidproduct_1095L,isbidproductrequest_292L,isdebitcard_729L,lastactivateddate_801D,lastapplicationdate_877D,lastapprcommoditycat_1041M,lastapprcommoditytypec_5251766M,lastapprcredamount_781A,lastapprdate_640D,lastcancelreason_561M,lastdelinqdate_224D,lastdependentsnum_448L,lastotherinc_902A,lastotherlnsexpense_631A,lastrejectcommoditycat_161M,lastrejectcommodtypec_5251769M,lastrejectcredamount_222A,lastrejectdate_50D,lastrejectreason_759M,lastrejectreasonclient_4145040M,lastrepayingdate_696D,lastst_736L,maininc_215A,mastercontrelectronic_519L,mastercontrexist_109L,maxannuity_159A,maxannuity_4075009A,maxdbddpdlast1m_3658939P,maxdbddpdtollast12m_3658940P,maxdbddpdtollast6m_4187119P,maxdebt4_972A,maxdpdfrom6mto36m_3546853P,maxdpdinstldate_3546855D,maxdpdinstlnum_3546846P,maxdpdlast12m_727P,maxdpdlast24m_143P,maxdpdlast3m_392P,maxdpdlast6m_474P,maxdpdlast9m_1059P,maxdpdtolerance_374P,maxinstallast24m_3658928A,maxlnamtstart6m_4525199A,maxoutstandbalancel12m_4187113A,maxpmtlast3m_4525190A,mindbddpdlast24m_3658935P,mindbdtollast24m_4525191P,mobilephncnt_593L,monthsannuity_845L,numactivecreds_622L,numactivecredschannel_414L,numactiverelcontr_750L,numcontrs3months_479L,numincomingpmts_3546848L,numinstlallpaidearly3d_817L,numinstls_657L,numinstlsallpaid_934L,numinstlswithdpd10_728L,numinstlswithdpd5_4187116L,numinstlswithoutdpd_562L,numinstmatpaidtearly2d_4499204L,numinstpaid_4499208L,numinstpaidearly3d_3546850L,numinstpaidearly3dest_4493216L,numinstpaidearly5d_1087L,numinstpaidearly5dest_4493211L,numinstpaidearly5dobd_4499205L,numinstpaidearly_338L,numinstpaidearlyest_4493214L,numinstpaidlastcontr_4325080L,numinstpaidlate1d_3546852L,numinstregularpaid_973L,numinstregularpaidest_4493210L,numinsttopaygr_769L,numinsttopaygrest_4493213L,numinstunpaid

In [23]:
# @ref: https://www.kaggle.com/code/ravi20076/homecredit-starter-training-v1/notebook Processing
feature_columns = train_df.drop(columns=["case_id", "WEEK_NUM", "target"], errors="ignore").columns
feature_ix_columns = [col + "_ix" for col in feature_columns]
category_columns = train_df.select_dtypes(include="object").columns

print(f"feature_columns: {len(feature_columns)}, category_columns: {len(category_columns)}")

In [30]:
# category to int
feature_d = {}

for col in feature_columns:
    vc = train_df[col].fillna("nan_value").astype(str).value_counts()
    d = {v: i for i, v in enumerate(vc.index, start=1)}
    # TODO: 代入先を新しいdfにする
    train_df[col+"_ix"] = np.vectorize(lambda v: d.get(v, 0))(train_df[col].fillna("nan_value").astype(str)).astype(np.int32)

    feature_d[col] = d

    del train_df[col]
    gc.collect()

feature_num_vocabs = [len(feature_d[v]) + 1 for v in feature_columns]

for col, num_vocabs in zip(feature_columns, feature_num_vocabs):
    print(col, num_vocabs)

KeyError: 'month_decision'

In [ ]:
# aggregate unique_feature
unique_train_df = train_df[feature_ix_columns].drop_duplicates()
unique_train_df["unique_ix"] = np.arange(unique_train_df.shape[0])

train_df = train_df.merge(unique_train_df, on=feature_ix_columns, how="left")
unique_feature = tf.convert_to_tensor(unique_train_df[feature_ix.columns].values)

In [ ]:
from src.dataloader import DataLoader
from src.models import DCNV2
from src.processor import Trainer, Predictor

dataloader = DataLoader(train_data, train_labels, unique_feature)
model = DCNV2(
    feature_num_vocabs=feature_num_vocabs,
    feat_dim=8,
    out_dim=32,
    num_cross=5,
    num_linear=0,
)
# Predictor不要かも
predictor = Predictor(out_dim=2)
trainer = Trainer(model, predictor)

SyntaxError: invalid syntax (2990837518.py, line 4)

### Training

In [ ]:
@tf.function(experimental_relax_shapes=True)
def forward_step(batch_inputs):
    with tf.GradientTape() as tape:
        loss, f1, acc = trainer(batch_inputs, training=True)

    gradients = tape.gradient(loss, trainer.trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainer.trainable_variables))

    return loss, f1, acc